# TensorFlow

## Creating a Model

### Load

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
tf.compat.v1.disable_eager_execution()

In [2]:
# Load training data set from CSV file
training_data_df = pd.read_csv("./03/sales_data_training.csv", dtype=float)

In [3]:
# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

In [4]:
# Load testing data set from CSV file
test_data_df = pd.read_csv('./03/sales_data_test.csv', dtype=float)

In [5]:
# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

In [6]:
# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0,1))
Y_scaler = MinMaxScaler(feature_range=(0,1))

In [7]:
# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

In [8]:
# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

In [9]:
print(X_scaled_testing.shape)
print(Y_scaled_testing.shape)

print("Note: Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(Y_scaler.scale_[0], Y_scaler.min_[0]))


(400, 9)
(400, 1)
Note: Y values were scaled by multiplying by 0.0000036968 and adding -0.1159


### Model

In [10]:
# Define model parameters
learning_rate = 0.001
training_epochs = 100
display_step = 5

In [11]:
# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

In [12]:
# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

In [13]:
# Section One: Define the layers of the neural network itself

# Input Layer
with tf.compat.v1.variable_scope('input'):
    X = tf.compat.v1.placeholder(tf.float32, shape=(None, number_of_inputs))

# Layer 1
with tf.compat.v1.variable_scope('layer_1'):
    weights = tf.compat.v1.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.initializers.GlorotUniform())
    biases = tf.compat.v1.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)


In [14]:
# Layer 2
with tf.compat.v1.variable_scope('layer_2'):
    weights = tf.compat.v1.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.initializers.GlorotUniform())
    biases = tf.compat.v1.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.compat.v1.variable_scope('layer_3'):
    weights = tf.compat.v1.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.initializers.GlorotUniform())
    biases = tf.compat.v1.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)


# Output Layer
with tf.compat.v1.variable_scope('output'):
    weights = tf.compat.v1.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.initializers.GlorotUniform())
    biases = tf.compat.v1.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)

In [15]:
# Section Two: Define the cost function of the neural network that will measure prediction accuracy during training
with tf.compat.v1.variable_scope('cost'):
    Y = tf.compat.v1.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.compat.v1.squared_difference(prediction, Y))

In [16]:
# Section Three: Define the optimizer function that will be run to optimize the neural network
with tf.compat.v1.variable_scope('train'):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cost)

In [17]:
# Summary operation to log progress of the network
with tf.compat.v1.variable_scope('logging'):
    tf.compat.v1.summary.scalar('current_cost', cost)
    summary = tf.compat.v1.summary.merge_all()

In [18]:
saver = tf.compat.v1.train.Saver()

### Model Training Loop

In [35]:
# Initialize a session so that we can run TensorFlow operations
with tf.compat.v1.Session() as session:
    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.compat.v1.global_variables_initializer())
    training_writer = tf.compat.v1.summary.FileWriter("./logs/training", session.graph)
    testing_writer  = tf.compat.v1.summary.FileWriter("./logs/testing", session.graph)   # Run the optimizer over and over to train the network.
   
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Print the current training status to the screen
        #print("Training pass: {}".format(epoch))
        if epoch % 5 == 0:
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y: Y_scaled_training})
            testing_cost,  testing_summary  = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
            print(epoch, training_cost, testing_cost)
            
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            


    # Training is now complete!
    print("Training is complete!")
    
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost  = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
    print("Final Training Cost: {}".format(final_training_cost))
    print("Final Testing Cost: {}".format(final_testing_cost))
    
    # Now that the neural network is trained, let's use it to make predictions for our test data.
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})
    
    
    # Unscale the data back to it's original units (dollars)
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)
    
    real_earnings = test_data_df['total_earnings'].values[0]
    predicted_earnings = Y_predicted[0][0]
    
    
    print("The actual earnings of Game #1 were ${}".format(real_earnings))
    print("Our neural network predicted earnings of ${}".format(predicted_earnings))
    
    
    save_path = saver.save(session, "logs/trained_model.ckpt")
    print("Model saved: {}".format(save_path))
        
    
    ##### Model builder saver #######
    model_builder = tf.compat.v1.saved_model.builder.SavedModelBuilder("exported_model")
    
    inputs = {
        'input': tf.compat.v1.saved_model.build_tensor_info(X)
    }
    outputs = {
        'earnings': tf.compat.v1.saved_model.build_tensor_info(prediction)
    }
    signature_def = tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
        inputs=inputs,
        outputs=outputs,
        method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME
    )
    
    model_builder.add_meta_graph_and_variables(
        session,
        [tf.compat.v1.saved_model.tag_constants.SERVING],
        signature_def_map={
            tf.compat.v1.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
        }
    )
    model_builder.save()
    
    

0 0.11288369 0.121972196
5 0.048956838 0.050865814
10 0.035772257 0.034390975
15 0.012850004 0.01361542
20 0.013623623 0.014966499
25 0.007049699 0.0073604668
30 0.006198388 0.0059101963
35 0.003893946 0.0037954056
40 0.0025634146 0.0024388412
45 0.002047818 0.0018512983
50 0.0016204517 0.0016477092
55 0.0011652529 0.0010561275
60 0.0008749021 0.0008701748
65 0.00069783966 0.0007044998
70 0.0005626246 0.00058712094
75 0.0004600473 0.0004918317
80 0.00038886498 0.00043496027
85 0.00032771588 0.0003894613
90 0.00027761646 0.0003518566
95 0.00023792508 0.0003229706
Training is complete!
Final Training Cost: 0.0002107830805471167
Final Testing Cost: 0.0002906522713601589
The actual earnings of Game #1 were $247537.0
Our neural network predicted earnings of $238626.328125
Model saved: logs/trained_model.ckpt
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: exported_model\saved_model.pb


### Load Saved Model

In [46]:
with tf.compat.v1.Session() as session:
    
    
    saver.restore(session, "logs/trained_model.ckpt")
    
    # Trained model loaded
    print("Trained model loaded from disk!")
    
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost  = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})
    print("Final Training Cost: {}".format(final_training_cost))
    print("Final Testing Cost: {}".format(final_testing_cost))
    
    # Now that the neural network is trained, let's use it to make predictions for our test data.
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})
    
    
    # Unscale the data back to it's original units (dollars)
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)
    
    real_earnings = test_data_df['total_earnings'].values[0]
    predicted_earnings = Y_predicted[0][0]
    
    
    print("The actual earnings of Game #1 were ${}".format(real_earnings))
    print("Our neural network predicted earnings of ${}".format(predicted_earnings))
    
    
    save_path = saver.save(session, "logs/trained_model.ckpt")
    print("Model saved: {}".format(save_path))


INFO:tensorflow:Restoring parameters from logs/trained_model.ckpt
Trained model loaded from disk!
Final Training Cost: 0.0002759622875601053
Final Testing Cost: 0.00042742499499581754
The actual earnings of Game #1 were $247537.0
Our neural network predicted earnings of $241256.609375
Model saved: logs/trained_model.ckpt
